# "Flat" Modeling for Gene Association with Lung Adenocarcinoma

Modeling with tabular data using 'classic' machine learning methods. The models aim to predict gene association with Lung Adenocarcinoma (LUAD). Data includes 'node features' (ontological features about the genes) and network features (a feature embedding on genes' position in the Protein-Protein Interaction (PPI) network). Genes are identified by their 'Ensembl' ID.

'Node features' come from the Human Protein Atlas, and the PPI network comes from the STRING dataset (restricted to human genes).

## Data and Setup

In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
data_path = 'data/HPAnode_PPInetwork_labels_v3.1.csv'
data = pd.read_csv(data_path, index_col=0)

In [3]:
data

,index,Gene,Gene synonym,Uniprot,Tissue RNA - lung [NX],Single Cell Type RNA - Mucus-secreting cells [NX],node_0,node_1,node_2,node_3,...,network_127,OMIM_pos,PROG_F_pos,PROG_UF_pos,CANCER_FPKM_pos,NIH_pos,DisGenNet_all_pos,DisGenNet_thresh_pos,Total_pos,training_label
Ensembl,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,13587,TSPAN6,"T245, TM4SF6, TSPAN-6",O43657,-0.293745,-0.037880,1.127839,0.280114,-0.562910,0.680988,...,0.046535,0,0,0,0,0,0,0,0,NaN
ENSG00000000419,3770,DPM1,"CDGIE, MPDS",O60762,0.526855,-0.037093,1.659597,1.147797,0.504771,-1.168400,...,-0.039454,0,0,0,0,0,0,0,0,NaN
ENSG00000000457,11231,SCYL3,"PACE-1, PACE1",Q8IZE3,-0.113110,-0.085092,0.917932,0.107147,-0.434965,-0.383316,...,-0.052639,0,0,0,0,0,0,0,0,NaN
ENSG00000000460,1709,C1orf112,FLJ10706,Q9NSG2,-0.587922,-0.103505,0.741257,-0.055523,-0.323032,0.007064,...,-0.048218,0,0,0,0,0,0,0,0,NaN
ENSG00000000938,4750,FGR,"c-fgr, p55c-fgr, SRC2",P09769,0.872642,-0.105079,1.660303,1.024769,0.060513,-0.997900,...,0.020117,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000284922,810,AP000812.5,0,0,-0.634371,-0.105079,0.700987,-0.087903,-0.319591,0.006010,...,-0.014959,0,0,0,0,0,0,0,0,NaN
ENSG00000285043,621,ALDOA,0,P04075,-0.603405,0.835389,2.009425,1.142514,0.587500,-1.108236,...,-0.069705,0,0,1,0,0,1,1,3,1.0
ENSG00000285188,121,AC008397.2,0,Q08493,-0.618888,-0.105079,1.292300,0.347360,-0.751088,-0.469017,...,-0.100453,0,0,0,0,0,0,0,0,NaN


In [4]:
label_col = 'training_label'
data[label_col] = data[label_col].astype('Int64')

data[label_col].value_counts()

0    191
1    191
Name: training_label, dtype: Int64

In [5]:
from sklearn.metrics import classification_report

def eval_model(model, X, y):
    preds = model.predict(X)
    clf_report = classification_report(y, preds, labels=[0, 1], target_names=['negative', 'positive'], digits=2)
    print(clf_report)

## Node-only Modeling

### Set up

In [6]:
num_node_feats = 100
node_feat_cols = ['Tissue RNA - lung [NX]', 'Single Cell Type RNA - Mucus-secreting cells [NX]'] + [f'node_{i}' for i in range(num_node_feats)]

# get subset of node features features + labels
node_data = data[node_feat_cols + [label_col]]

# restrict to data with labels
node_data_labeled = node_data[node_data[label_col].notna()]
node_data_labeled

,Tissue RNA - lung [NX],Single Cell Type RNA - Mucus-secreting cells [NX],node_0,node_1,node_2,node_3,node_4,node_5,node_6,node_7,...,node_91,node_92,node_93,node_94,node_95,node_96,node_97,node_98,node_99,training_label
Ensembl,,,,,,,,,,,,,,,,,,,,,
ENSG00000005102,-0.660176,-0.105079,2.097750,-1.520459,0.979743,0.102077,-0.064198,-0.095211,0.785779,0.626443,...,0.099556,0.097325,-0.352616,0.111681,-0.197486,-0.071428,0.550138,0.093125,-0.146437,0
ENSG00000005339,0.150101,-0.069827,2.622879,0.092524,1.558535,-1.148822,0.606971,0.573626,0.106728,-0.357630,...,-0.327284,-0.087676,0.254183,-0.066311,-0.014220,-0.059492,0.095315,0.159288,-0.186821,1
ENSG00000006282,-0.144076,-0.055349,1.143110,-0.375444,-0.293915,0.119900,-0.075660,-0.116266,-0.127803,-0.645784,...,0.001051,-0.058577,-0.081520,-0.095410,0.039419,0.000573,0.150117,-0.226045,0.051211,0
ENSG00000010610,0.516533,-0.105079,1.114144,0.627415,-0.097759,1.059575,0.282671,0.423967,-0.340315,0.361438,...,-0.009328,-0.113221,-0.089241,-0.018135,-0.101621,-0.063740,-0.087179,-0.009082,0.027355,0
ENSG00000010810,0.578465,-0.101302,1.677973,0.581332,-0.799185,-0.650435,0.386856,0.852724,0.152394,0.765213,...,-0.085160,0.043194,0.094463,-0.135246,0.052647,0.171408,0.179968,-0.194070,0.171541,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000266028,-0.598244,-0.102718,1.523235,-0.099984,-0.281082,-0.048439,-0.502699,-0.269441,0.803980,-0.275743,...,-0.008828,0.055448,0.039220,-0.028579,-0.077158,0.033021,-0.021508,-0.024827,-0.047601,0
ENSG00000272514,-0.598244,-0.105079,1.057383,-0.401744,-0.391620,0.129470,-0.026114,-0.302946,-0.056680,-0.614087,...,-0.056147,0.078024,-0.046593,-0.103124,-0.125405,-0.000180,-0.054954,0.099468,-0.111486,0
ENSG00000276203,-0.655015,-0.104449,1.035287,-0.416219,-0.386367,0.113579,-0.004378,-0.294410,-0.050775,-0.610978,...,0.004381,-0.003438,0.007235,-0.006817,-0.000525,0.002603,-0.005936,0.000313,-0.001213,0


In [7]:
# separate features and labels
node_feats = node_data_labeled[node_feat_cols]
node_labels = node_data_labeled[label_col].astype('int32')

In [8]:
# create train-test split

from sklearn.model_selection import train_test_split
test_size = 0.25

X_train, X_test, y_train, y_test = train_test_split(node_feats, node_labels, test_size=test_size, shuffle=True, stratify=node_labels, random_state=360)
# NOTE: train test split is shuffled and stratified across labels

### Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# parameter grid to search over
parameters = {
                'n_estimators':[20, 50, 100, 150, 200],
                'criterion':['gini'],
                'max_depth':[1, 2, 3, 4, 5]
             }

# base random forest model
rf = RandomForestClassifier(n_jobs=-1)

# perform a gridsearch with 5-fold crossvalidation to find the best model
rf_clf = GridSearchCV(rf, parameters, n_jobs=-1, refit=True, cv=5, return_train_score=True)

rf_clf.fit(X_train, y_train)


# show choice of parameters that yielded the best performance
print('Best Parameters')
print(rf_clf.best_params_)
print('\n')

# evaluate model
print('Training Metrics')
eval_model(rf_clf.best_estimator_, X_train, y_train)

print()
print('Testing Metrics')
eval_model(rf_clf.best_estimator_, X_test, y_test)

Best Parameters
{'criterion': 'gini', 'max_depth': 2, 'n_estimators': 50}


Training Metrics
              precision    recall  f1-score   support

    negative       0.81      0.85      0.83       143
    positive       0.84      0.80      0.82       143

    accuracy                           0.83       286
   macro avg       0.83      0.83      0.83       286
weighted avg       0.83      0.83      0.83       286


Testing Metrics
              precision    recall  f1-score   support

    negative       0.63      0.71      0.67        48
    positive       0.67      0.58      0.62        48

    accuracy                           0.65        96
   macro avg       0.65      0.65      0.64        96
weighted avg       0.65      0.65      0.64        96



## Network-only Modeling

### Set up

In [10]:
num_network_feats = 128
network_feat_cols = [f'network_{i}' for i in range(num_node_feats)]

# get subset of node features features + labels
network_data = data[network_feat_cols + [label_col]]

# restrict to data with labels
network_data_labeled = network_data[network_data[label_col].notna()]
network_data_labeled

,network_0,network_1,network_2,network_3,network_4,network_5,network_6,network_7,network_8,network_9,...,network_91,network_92,network_93,network_94,network_95,network_96,network_97,network_98,network_99,training_label
Ensembl,,,,,,,,,,,,,,,,,,,,,
ENSG00000005102,-0.184647,-0.127033,0.052034,0.182896,0.119084,-0.331242,0.184941,-0.026008,-0.148555,0.144442,...,0.197153,-0.025282,0.024105,-0.023171,0.107037,0.179881,-0.061453,-0.137580,0.193667,0
ENSG00000005339,-0.063438,-0.123012,-0.025039,0.447147,0.187767,-0.070443,-0.044440,-0.194156,0.062328,0.117881,...,0.191070,-0.010945,0.051817,-0.096172,0.099404,0.052227,0.107331,-0.030173,0.059622,1
ENSG00000006282,-0.042194,-0.015269,0.025426,0.159128,0.116125,-0.191182,-0.105877,-0.064388,-0.085375,-0.006664,...,0.057874,-0.163383,0.158717,0.020384,-0.109713,0.029926,0.056307,-0.208349,0.187577,0
ENSG00000010610,0.049363,-0.134285,0.152762,0.164370,0.240288,0.016218,-0.035477,-0.036192,0.007737,0.191897,...,-0.164296,-0.072100,-0.062973,-0.279423,0.209841,-0.319872,0.178506,0.125086,-0.033573,0
ENSG00000010810,0.031518,-0.010374,0.257410,-0.026564,0.118352,0.005907,0.084970,-0.128120,0.005545,0.196510,...,-0.062661,-0.004803,0.012089,-0.027547,0.172744,0.049692,0.270915,0.095332,0.001077,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000266028,-0.084297,0.084494,0.157879,-0.144265,-0.068077,-0.064292,-0.049912,-0.088345,-0.014282,0.077820,...,0.021764,-0.121634,0.090939,0.096539,-0.019622,0.079354,0.080837,-0.019187,0.099724,0
ENSG00000272514,-0.094307,-0.088711,0.010982,-0.113788,0.016688,-0.101340,0.033163,-0.132528,-0.069357,0.017188,...,0.058968,-0.104994,0.137027,-0.014768,-0.018511,-0.054909,0.037750,-0.140715,0.154080,0
ENSG00000276203,0.045890,-0.102354,0.043295,-0.015650,0.124472,-0.108613,0.105846,-0.256952,0.015107,0.040565,...,0.086761,0.089809,0.211379,-0.088506,-0.012749,0.088272,0.205796,-0.081693,0.034378,0


In [11]:
# separate features and labels
network_feats = network_data_labeled[network_feat_cols]
network_labels = network_data_labeled[label_col].astype('int32')

In [12]:
# create train-test split

from sklearn.model_selection import train_test_split
test_size = 0.25

X_train, X_test, y_train, y_test = train_test_split(network_feats, network_labels, test_size=test_size, shuffle=True, stratify=network_labels)
# NOTE: train test split is shuffled and stratified across labels

### Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# parameter grid to search over
parameters = {
                'n_estimators':[20, 50, 100, 150, 200],
                'criterion':['gini'],
                'max_depth':[1, 2, 3, 4, 5]
             }

# base random forest model
rf = RandomForestClassifier(n_jobs=-1)

# perform a gridsearch with 5-fold crossvalidation to find the best model
rf_clf = GridSearchCV(rf, parameters, n_jobs=-1, refit=True, cv=5, return_train_score=True)

rf_clf.fit(X_train, y_train)


# show choice of parameters that yielded the best performance
print('Best Parameters')
print(rf_clf.best_params_)
print('\n')

# evaluate model
print('Training Metrics')
eval_model(rf_clf.best_estimator_, X_train, y_train)

print()
print('Testing Metrics')
eval_model(rf_clf.best_estimator_, X_test, y_test)

Best Parameters
{'criterion': 'gini', 'max_depth': 5, 'n_estimators': 150}


Training Metrics
              precision    recall  f1-score   support

    negative       0.95      0.97      0.96       143
    positive       0.97      0.95      0.96       143

    accuracy                           0.96       286
   macro avg       0.96      0.96      0.96       286
weighted avg       0.96      0.96      0.96       286


Testing Metrics
              precision    recall  f1-score   support

    negative       0.74      0.73      0.74        48
    positive       0.73      0.75      0.74        48

    accuracy                           0.74        96
   macro avg       0.74      0.74      0.74        96
weighted avg       0.74      0.74      0.74        96



## Node + Network Modeling

### Set up

In [14]:
node_network_feat_cols = node_feat_cols + network_feat_cols

# get subset of node features features + labels
node_network_data = data[node_network_feat_cols + [label_col]]

# restrict to data with labels
node_network_data_labeled = node_network_data[node_network_data[label_col].notna()]
node_network_data_labeled

,Tissue RNA - lung [NX],Single Cell Type RNA - Mucus-secreting cells [NX],node_0,node_1,node_2,node_3,node_4,node_5,node_6,node_7,...,network_91,network_92,network_93,network_94,network_95,network_96,network_97,network_98,network_99,training_label
Ensembl,,,,,,,,,,,,,,,,,,,,,
ENSG00000005102,-0.660176,-0.105079,2.097750,-1.520459,0.979743,0.102077,-0.064198,-0.095211,0.785779,0.626443,...,0.197153,-0.025282,0.024105,-0.023171,0.107037,0.179881,-0.061453,-0.137580,0.193667,0
ENSG00000005339,0.150101,-0.069827,2.622879,0.092524,1.558535,-1.148822,0.606971,0.573626,0.106728,-0.357630,...,0.191070,-0.010945,0.051817,-0.096172,0.099404,0.052227,0.107331,-0.030173,0.059622,1
ENSG00000006282,-0.144076,-0.055349,1.143110,-0.375444,-0.293915,0.119900,-0.075660,-0.116266,-0.127803,-0.645784,...,0.057874,-0.163383,0.158717,0.020384,-0.109713,0.029926,0.056307,-0.208349,0.187577,0
ENSG00000010610,0.516533,-0.105079,1.114144,0.627415,-0.097759,1.059575,0.282671,0.423967,-0.340315,0.361438,...,-0.164296,-0.072100,-0.062973,-0.279423,0.209841,-0.319872,0.178506,0.125086,-0.033573,0
ENSG00000010810,0.578465,-0.101302,1.677973,0.581332,-0.799185,-0.650435,0.386856,0.852724,0.152394,0.765213,...,-0.062661,-0.004803,0.012089,-0.027547,0.172744,0.049692,0.270915,0.095332,0.001077,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000266028,-0.598244,-0.102718,1.523235,-0.099984,-0.281082,-0.048439,-0.502699,-0.269441,0.803980,-0.275743,...,0.021764,-0.121634,0.090939,0.096539,-0.019622,0.079354,0.080837,-0.019187,0.099724,0
ENSG00000272514,-0.598244,-0.105079,1.057383,-0.401744,-0.391620,0.129470,-0.026114,-0.302946,-0.056680,-0.614087,...,0.058968,-0.104994,0.137027,-0.014768,-0.018511,-0.054909,0.037750,-0.140715,0.154080,0
ENSG00000276203,-0.655015,-0.104449,1.035287,-0.416219,-0.386367,0.113579,-0.004378,-0.294410,-0.050775,-0.610978,...,0.086761,0.089809,0.211379,-0.088506,-0.012749,0.088272,0.205796,-0.081693,0.034378,0


In [15]:
# separate features and labels
node_network_feats = node_network_data_labeled[node_network_feat_cols]
node_network_labels = node_network_data_labeled[label_col].astype('int32')

In [16]:
# create train-test split

from sklearn.model_selection import train_test_split
test_size = 0.25

X_train, X_test, y_train, y_test = train_test_split(node_network_feats, node_network_labels, test_size=test_size, shuffle=True, stratify=node_network_labels)
# NOTE: train test split is shuffled and stratified across labels

### Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# # parameter grid to search over
# parameters = {
#                 'n_estimators':[20, 50, 100, 150, 200],
#                 'criterion':['gini', 'entropy'],
#                 'max_depth':[1, 2, 3, 4, 5]
#              }

# # base random forest model
# rf = RandomForestClassifier(n_jobs=-1)

# # perform a gridsearch with 5-fold crossvalidation to find the best model
# rf_clf = GridSearchCV(rf, parameters, n_jobs=-1, refit=True, cv=5, return_train_score=True)

rf_clf = RandomForestClassifier(n_estimators=150, max_depth=3, criterion='gini', n_jobs=-1)
rf_clf.fit(X_train, y_train)


# # show choice of parameters that yielded the best performance
# print('Best Parameters')
# print(rf_clf.best_params_)
# print('\n')

# evaluate model
print('Training Metrics')
eval_model(rf_clf, X_train, y_train)

print()
print('Testing Metrics')
eval_model(rf_clf, X_test, y_test)

Training Metrics
              precision    recall  f1-score   support

    negative       0.90      0.93      0.92       143
    positive       0.93      0.90      0.91       143

    accuracy                           0.92       286
   macro avg       0.92      0.92      0.92       286
weighted avg       0.92      0.92      0.92       286


Testing Metrics
              precision    recall  f1-score   support

    negative       0.75      0.81      0.78        48
    positive       0.80      0.73      0.76        48

    accuracy                           0.77        96
   macro avg       0.77      0.77      0.77        96
weighted avg       0.77      0.77      0.77        96

